In [1]:
import akshare as ak


def get_trade(symbol):
    """
    获取行业信息
    :param symbol:
    :return:
    """
    df = ak.stock_individual_info_em(symbol=symbol)
    dct = dict(zip(df['item'], df['value']))
    return dct['行业'], dct['股票简称']

get_trade('003004')

('计算机设备', '声迅股份')

In [2]:
from pyecharts import options as opts
from pyecharts.charts import Kline, Line, Grid
from pyecharts.globals import CurrentConfig, NotebookType
from IPython.display import display
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

data = [
    [2320.26, 2320.26, 2287.3, 2362.94],
    [2300, 2291.3, 2288.26, 2308.38],
    [2295.35, 2346.5, 2295.35, 2345.92],
    [2347.22, 2358.98, 2337.35, 2363.8],
    [2360.75, 2382.48, 2347.89, 2383.76],
]

def plot_kline(df, symbol):
    date_list = df['日期'].to_list()
    ndf = df[['开盘', '收盘', '最低', '最高']]
    data = ndf.values.tolist()
    
    area, name = get_trade(symbol)
    
    kline = (
    Kline()
    .add_xaxis(date_list)
    .add_yaxis(name, data)
    .set_global_opts(
        yaxis_opts=opts.AxisOpts(
            is_scale=True,
            splitarea_opts=opts.SplitAreaOpts(
                is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
            ),
        ),
        xaxis_opts=opts.AxisOpts(is_scale=True),
        datazoom_opts=[opts.DataZoomOpts(type_="inside")],
        title_opts=opts.TitleOpts(title=symbol),
    ))
    
    line = (
        Line()
        .add_xaxis(xaxis_data=date_list)
        .add_yaxis(
            series_name="MA5",
            y_axis=df['ma_5'].tolist(),
            is_smooth=True,
            is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=1.5, opacity=0.8, color="#4B4453"),
            label_opts=opts.LabelOpts(is_show=False),
        )
        .add_yaxis(
            series_name="MA10",
            y_axis=df['ma_10'].tolist(),
            is_smooth=True,
            is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=1.5, opacity=0.8, color="#FF8C00"),
            label_opts=opts.LabelOpts(is_show=False),
        )
        .add_yaxis(
            series_name="MA20",
            y_axis=df['ma_20'].tolist(),
            is_smooth=True,
            is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=1.5, opacity=0.8, color="#4B0082"),
            label_opts=opts.LabelOpts(is_show=False),
        )
        .set_global_opts(xaxis_opts=opts.AxisOpts(type_="category"))
    )
    
    # Kline And Line
    overlap_kline_line = kline.overlap(line)

    # Grid Overlap + Bar
    grid_chart = Grid(
        init_opts=opts.InitOpts(
            width="900px",
            height="800px",
            animation_opts=opts.AnimationOpts(animation=False),
        )
    )
    grid_chart.add(
        overlap_kline_line,
        grid_opts=opts.GridOpts(pos_left="10%", pos_right="8%", height="50%"),
    )

    grid_chart.load_javascript()
    return grid_chart.render_notebook()
    

# display(plot_kline(data, '003004'))

In [3]:
import os
import time
import pandas as pd

# 8, 13, 21日均线

folder = 'daily_data/'
files = os.listdir(folder)

count = 0
today = time.localtime(time.time()).tm_mday

exclude = ['600317', '603206']

for f in files[:4000]:
    try:
        
        if not f.endswith('.csv'):
            continue
        
        path = os.path.join(folder, f)
        strftime = os.path.getmtime(path)
        
        #if time.localtime(strftime).tm_mday != today:
        #    continue
            
        df = pd.read_csv(path)

        df['max_ma'] = df[['ma_5', 'ma_10', 'ma_20']].max(axis=1)
        df['min_ma'] = df[['ma_5', 'ma_10', 'ma_20']].min(axis=1)
        df['line'] = df['max_ma']/df['min_ma']
        
        # 引入三天的均值作为基线
        df['mean_line'] = df['line'].rolling(3).mean() 
        
        # 获取实体最大偏离绝对值，不能偏离均线均值太远
        df['ma_mean'] = df[['ma_5', 'ma_10', 'ma_20']].mean(axis=1)
        def max_deviate(ser):
            # print(ser)
            dev1 = abs((ser['开盘'] - ser['ma_mean'])/ser['昨收'] * 100)
            dev2 = abs((ser['收盘'] - ser['ma_mean'])/ser['昨收'] * 100)
            return max(dev1, dev2)
        df['ma_deviate'] = df.apply(max_deviate, axis=1)

        df['deviate_mean'] = df['ma_deviate'].rolling(3).mean() 
        
        # 计算是否连续三天阴线
        df['is_green'] = df['收盘'] < df['开盘']
        df['3_days_false'] = df[['is_green']].rolling(3).apply(lambda x: all(x))
        
        data = df.tail(1).to_dict(orient='records')[0]
        
        cond1 = data['mean_line'] <= 1.010
        cond2 = data['收盘'] >= data['ma_60']
        cond3 = data['deviate_mean'] <= 3
        cond4 = data['ma_5'] >= data['ma_20']*0.997 # 五日线不能低于20日线太多
        cond5 = data['换手率'] >= 0.85  # 换手率太低 交易不活跃
        cond6 = not data['3_days_false']
        #print(data['ma_5'], data['ma_20'])
        
        if cond1 and cond2 and cond3 and cond4 and cond5 and cond6:
            symbol, _ = f.split('.')
            
            if symbol in exclude:
                continue
            
            ndf = df[-80:]
            graph = plot_kline(ndf, symbol)
            display(graph)
            
            print(ndf.tail()[['日期', 'mean_line', 'deviate_mean', 'ma_deviate']])

        count += 1
        # break

    except Exception as e:
        print(e)
        print(f)
        
# 价格覆盖线  000561
# 下降趋势过滤掉

FileNotFoundError: [Errno 2] No such file or directory: 'daily_data/'

In [ ]:
portfolio